In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing libraries:- CUDA,CV2, matplotlib, sklearn etc**

In [2]:
import cudf as cd
import cupy as cp
import glob
import cv2
from IPython.display import clear_output
from matplotlib import pyplot as plt
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import roc_auc_score,confusion_matrix,mean_squared_log_error,precision_recall_curve
from sklearn.metrics import classification_report


**Setting the height and weight of the image**

In [3]:
from PIL import Image
from warnings import filterwarnings
filterwarnings("ignore")

HEIGHT = 150
WEIGHT = 150

**Path to the dataset**

In [4]:
path = "../input/face-mask-12k-images-dataset/Face Mask Dataset/"
os.listdir(path)

**Adding the images to the dataframe**

In [5]:
image_path = []
label = []
location = []

for where in os.listdir(path):
    for status in os.listdir(path+"/"+where):
        for image in glob.glob(path+where+"/"+status+"/"+"*.png"):
            image_path.append(image)
            label.append(status)
            location.append(where)
            
data = cd.DataFrame({'image_path':image_path,'label':label,'location':location})
data.head()

**Splitting the dataset into train, test and validation **

In [6]:
train_data =data[data['location']=='Train'].sample(frac=1)
test_data = data[data['location']=='Test'].sample(frac=1)
validation_data = data[data['location']=='Validation'].sample(frac=1)

Preprocessing the data using the following steps:
1. Declaring the X and y variables
2. Reading images into an array
3. Labeling these images
4. Normalizing the X values

In [7]:
def data_preprocessing(input_data):
    input_data = input_data.reset_index().drop("index", axis = 1)
    X = []
    y = []
    for idx,i in enumerate(range(len(input_data))):
        if idx%500 == 0:
            print(f'{idx} completed')
        img_array = cv2.imread(input_data["image_path"][i], cv2.IMREAD_GRAYSCALE)
        new_image_array = cv2.resize(img_array, (150, 150))
        X.append(new_image_array)
        if 'label' in input_data.columns:
            if input_data["label"][i] == "WithMask":
                y.append(1)
            else:
                y.append(0)
    clear_output()
    del img_array
    del new_image_array
    X = cp.array(X,dtype='float32').reshape(len(input_data),-1)
    y = cp.array(y).reshape(-1,1)
    X = X/255.0
    return X,y

**%%time is used due to large amount of data and storing the preprocessed data**

In [8]:
%%time
Train_data,Train_labels = data_preprocessing(train_data)
Validation_data, Validation_labels = data_preprocessing(validation_data)
Test_data,_ = data_preprocessing(test_data)

**Checking the GPU memory**

In [9]:
!nvidia-smi

**Importing keras from tensorflow**

In [10]:
from tensorflow import keras
from tensorflow.keras import layers,callbacks
from keras.metrics import MSLE

**Importing model metrics and hyper-parameters**

In [11]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, SpatialDropout2D, BatchNormalization, Input, Activation, Dense, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import plot_model
from keras.losses import binary_crossentropy


**Building the CNN model**

In [12]:
model = Sequential()
model.add(Input(shape=(HEIGHT,WEIGHT,1,)))
model.add(Conv2D(filters=16,kernel_size=(2,2),padding="same"))
model.add(Activation("relu"))
model.add(SpatialDropout2D(0.25))
model.add(MaxPool2D(pool_size=(4,4)))
model.add(Conv2D(filters=32,kernel_size=(2,2),padding="same"))
model.add(Activation("relu"))
model.add(SpatialDropout2D(0.25))
model.add(MaxPool2D(pool_size=(4,4),strides=(4,4)))
model.add(Flatten())
model.add(Dense(2048))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))
optimizer = Adam(lr=0.001)
model.compile(optimizer = optimizer ,metrics=["accuracy"], loss = binary_crossentropy)

In [13]:
model.summary()

In [14]:
plot_model(model,show_shapes=True,show_layer_names=True)

**Normalizing X_train, y_train, X_test, y_test**

In [15]:
X_train = Train_data.reshape((-1,150,150,1)).get()
X_test = Validation_data.reshape((-1,150,150,1)).get()
y_train = Train_labels.flatten().get()
y_test = Validation_labels.flatten().get()

In [16]:
print(X_train.shape)
print(y_train.shape)


**Fitting the CNN model using Adam optimizer**

In [17]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),
                    batch_size = 32, epochs=25)

**Storing the predicted values in y_pred**

In [18]:
y_pred=cp.array(model.predict(X_test),dtype='int32').flatten()
y_test=cp.array(y_test, dtype='int32').flatten()
y_pred.shape,y_test.shape

In [19]:
print(history.history['accuracy'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()